In [1]:
!pip install opencv-python opencv-contrib-python numpy matplotlib pillow tqdm

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


In [48]:
# # rename files in folder
import os
import re
import shutil

folder = './files'
for filename in os.listdir(folder):
    test_re = re.compile(r'\((\d*)\).png')
    match = test_re.search(filename)
    print(match)
    if match:
        new_name = match.group(1) + '.png'
        print(filename, '->', new_name)
        os.rename(os.path.join(folder, filename), os.path.join(folder, new_name))

<re.Match object; span=(1, 8), match='(1).png'>
 (1).png -> 1.png
<re.Match object; span=(1, 9), match='(10).png'>
 (10).png -> 10.png
<re.Match object; span=(1, 10), match='(100).png'>
 (100).png -> 100.png
<re.Match object; span=(1, 10), match='(101).png'>
 (101).png -> 101.png
<re.Match object; span=(1, 10), match='(102).png'>
 (102).png -> 102.png
<re.Match object; span=(1, 10), match='(103).png'>
 (103).png -> 103.png
<re.Match object; span=(1, 10), match='(104).png'>
 (104).png -> 104.png
<re.Match object; span=(1, 10), match='(105).png'>
 (105).png -> 105.png
<re.Match object; span=(1, 10), match='(106).png'>
 (106).png -> 106.png
<re.Match object; span=(1, 10), match='(107).png'>
 (107).png -> 107.png
<re.Match object; span=(1, 10), match='(108).png'>
 (108).png -> 108.png
<re.Match object; span=(1, 10), match='(109).png'>
 (109).png -> 109.png
<re.Match object; span=(1, 9), match='(11).png'>
 (11).png -> 11.png
<re.Match object; span=(1, 10), match='(110).png'>
 (110).png -> 1

In [55]:
from tqdm.notebook import tqdm
#%matplotlib inline

from numpy import ndarray, dtype, generic
from matplotlib import pyplot as plt

import numpy as np
import os
import cv2

folder_base = './files'

IMG_END = 224

output_folder = './output'
count = 0

head_offset = 150
foot_offset = 300
safe_offset = 50

slice_length = 80

# clear output folder
for filename in tqdm(os.listdir(output_folder), desc='clearing output folder'):
    os.remove(os.path.join(output_folder, filename))


def get_output_filename():
    global count
    count += 1
    return os.path.join(output_folder, '{}.png'.format(count))


# def check_part(part_img: ndarray[int, dtype[generic]]):
#     hsv = cv2.cvtColor(part_img, cv2.COLOR_BGR2HSV)
#     lower = np.array([0, 0, 0])
#     upper = np.array([255, 255, 90])
#     mask = cv2.inRange(hsv, lower, upper)
#
#     cv2.imshow('mask', mask)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
#
#     res = cv2.bitwise_and(hsv, hsv, mask=mask)
#     value = cv2.split(res)[2]
#     r_num = 0
#     for index in range(value.shape[0]):
#         for j in range(value.shape[1]):
#             if value[index, j] > 0:
#                 r_num += 1
#
#     if r_num > 10:
#         return True
#     else:
#         return False


def check_part(part_img: ndarray[int, dtype[generic]]):
    img_gray = cv2.cvtColor(part_img, cv2.COLOR_BGR2GRAY)
    img_bq = cv2.medianBlur(img_gray, 1)
    _, output1 = cv2.threshold(img_bq, 140, 255, cv2.THRESH_BINARY)
    r_num = np.count_nonzero(output1 != 255)
    # print(r_num)
    if r_num > 100:  # well, just fucking experience number
        return True
    else:
        return False


for i in tqdm(range(1, IMG_END + 1), desc='processing page'):
    target_file = os.path.join(folder_base, str(i) + '.png')
    img = cv2.imread(target_file)
    # get image size
    height, width, channel = img.shape

    part_start = -1
    part_end = -1

    for scan_start in range(head_offset, height - foot_offset, slice_length):
        scan_end = scan_start + slice_length
        if scan_end > height - foot_offset:
            scan_end = height - foot_offset
        part = img[scan_start:scan_end, :]
        should = check_part(part)
        if should:
            if part_start == -1:
                part_start = scan_start
        else:
            if part_start != -1:
                part_end = scan_start
                save_img = img[part_start - safe_offset:part_end + safe_offset, :]
                num = get_output_filename()
                cv2.imwrite(num, save_img)
                part_start = -1
                part_end = -1

    if part_start != -1:
        part_end = height - head_offset
        save_img = img[part_start - 100:part_end + 100, :]
        num = get_output_filename()
        cv2.imwrite(num, save_img)

clearing output folder:   0%|          | 0/659 [00:00<?, ?it/s]

processing page:   0%|          | 0/224 [00:00<?, ?it/s]